# Preprocessing intracranial EEG using MNE-python


*WIRED 2024*  
[Maansi Desai, PhD](https://maansidesai.github.io/)  
Postdoctoral Researcher in the [Hamilton Lab](https://slhs.utexas.edu/research/hamilton-lab)
<br>
Department of Speech, Language, and Hearing Sciences 
<br>
The University of Texas at Austin 

## Part 3: Plotting electrodes on brain using MNE brain.viz function


In [ ]:
# %matplotlib notebook

import mne
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from mne_bids import read_raw_bids, print_dir_tree, BIDSPath
from mne_bids.path import get_bids_path_from_fname
from bids import BIDSLayout
from ecog_preproc_utils import transformData
import bids 
import re  # regex for comparing channel names

## Download BIDS iEEG dataset

Here we will download an example iEEG dataset from [this dataset from the Hamilton lab](https://openneuro.org/datasets/ds004993/versions/1.1.1). If you have ran [`01_ieeg_preprocessing_MNE`](01_ieeg_preprocessing_MNE) and [`02_ieeg_preprocessing_MNE_epochs.ipynb`](02_ieeg_preprocessing_MNE_epochs.ipynb), then you should already have downloaded the necessary datasets locally. 

For this notebook we will use data from: 
 - `sub-W1`, `iemu`, `B3`, `movietrailers`
 - `sub-W2`, `iemu`, `B8`, `timit5`
 - `sub-W3`, `iemu`, `B8`, `timit4`

In [ ]:
# This is the example participant's data that we will load for the tutorial,
# but there are more options.
subj = 'W2'
sess = 'iemu'
task = 'timit5'
acq = 'B8'
run = '01'

In [ ]:
import mne_bids
mne_bids.__version__

In [ ]:
# Change the data directory below to where your data are located. 
parent_dir = 'data/ds004993'  # This is on your local machine

bids_path = BIDSPath(
    root=parent_dir, subject=subj, session=sess, task=task, acquisition=acq, run=run, datatype="ieeg"
)
print(bids_path)

## BIDS layout

We can use `pybids` to show a little bit about the files in this BIDS dataset. We won't get as much into this, but if you'd like to try this tutorial on your own you may wish to delve into this more.

In [ ]:
layout = BIDSLayout(parent_dir)

In [ ]:
layout.get_tasks()

In [ ]:
all_files = layout.get()
print("There are {} files in the layout.".format(len(all_files)))
print("\nThe first 10 files are:")
all_files[:10]

In [ ]:
print_dir_tree(parent_dir, max_depth=3)

## Let's load some iEEG data!

In [ ]:
# Read data and extract parameters from BIDS files
raw = read_raw_bids(bids_path, verbose=True)

# Electrodes on the brain


In [ ]:
brain_kwargs = dict(alpha=0.1, background="white", view='split',cortex="low_contrast")
brain = mne.viz.Brain(f'sub-{subj}', surf='pial', subjects_dir=f'{parent_dir}/freesurfer', alpha=0.5,
                      background='white')
trans = mne.read_trans(f'{parent_dir}/freesurfer/sub-{subj}/bem/sub-{subj}_head-trans.fif')
brain.add_sensors(raw.info, trans=trans)